# **1. Kaggle**

* 전 세계 데이터 사이언티스트들이 데이터를 분석할 수 있도록 대회를 개최
* 데이터를 분석하고 토론할 수 있는 커뮤니티 제공
* 데이터 셋, 파이썬 자료, R 자료를 제공

In [1]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv("https://bit.ly/fc-ml-titanic")

In [5]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* PassengerId	: 승객 아이디
* Survived : 생존 여부(1:생존, 0:사망)
* Pclass : 등급
* Name : 승객 이름
* Sex	: 성별
* Age	: 나이
* SibSp	: 형제, 자매, 배우자 수
* Parch	: 부모, 자식 수
* Ticket	: 티켓 번호
* Fare	: 요금
* Cabin	: 좌석 번호
* Embarked : 탑승 항구

# **2. 전처리**

In [7]:
# 전처리
# train/ validation 세트로 나누기
# 1. feature와 label을 정의
# 2. 적절한 비율로 train/validation set를 나눔
feature = ['Pclass', 'Sex', 'Age', 'Fare'] #재료
label = ['Survived'] # 결과

In [8]:
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [9]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [10]:
from sklearn.model_selection import train_test_split

* test_size : validation set에 할당할 비율(0.2 -> 20$)
* shuffle : 셔플 옵션(기본 true)
* random_state : 랜덤 시드값

In [14]:
x_train, x_valid, y_train, y_valid = train_test_split(train[feature], train[label], test_size=0.2, shuffle=True, random_state=10)


In [15]:
x_train.shape, y_train.shape

((712, 4), (712, 1))

In [16]:
x_valid.shape, y_valid.shape

((179, 4), (179, 1))

# **3. 결측치 처리**

In [17]:
train.info() # 결측치 확인 (Age, Cabin, Embarked)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [20]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [22]:
train['Age'].isnull().sum()

177

In [23]:
# 결측치를 0으로 처리
train['Age'].fillna(0).describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [25]:
# 결측치를 평균나이로 처리
train['Age'].fillna(train['Age'].mean()).describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

### **3-1. impute**

* 2개 이상의 column을 한 번에 처리할때 사용
* [impute document](https://scikit-learn.org/stable/modules/impute.html
)


In [26]:
from sklearn.impute import SimpleImputer

In [27]:
imputer = SimpleImputer(strategy='mean')

In [28]:
# fit()를 통해 결측치에 대한 학습을 진행
imputer.fit(train[['Age','Pclass']])

SimpleImputer()

In [29]:
# transform()은 실제 결측치에 대한 처리를 해주는 함수
result = imputer.transform(train[['Age','Pclass']])

In [30]:
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [31]:
train[['Age','Pclass']] = result

In [32]:
train[['Age', 'Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [33]:
train[['Age', 'Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


### **3-2. Categorical Column 데이터에 대한 결측치 처리**

In [34]:
train = pd.read_csv("https://bit.ly/fc-ml-titanic")

In [35]:
train['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [36]:
imputer = SimpleImputer(strategy='most_frequent')

In [37]:
result = imputer.fit_transform(train[['Embarked','Cabin']])

In [39]:
train[['Embarked','Cabin']] = result

In [40]:
train[['Embarked','Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64

# **4. Lable Encoding**

* 문자(categorical)를 수치(numerical)로 변환

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        891 non-null    object 
 11  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [52]:
def convert(data):
  if data =='male':
    return 1
  elif data == 'female':
    return 0

In [53]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [54]:
train['Sex'].apply(convert)

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [55]:
from sklearn.preprocessing import LabelEncoder

In [56]:
le = LabelEncoder()

In [57]:
train['Sex_num'] = le.fit_transform(train['Sex'])

In [59]:
train['Sex_num'].value_counts()

1    577
0    314
Name: Sex_num, dtype: int64

In [61]:
le.classes_

array(['female', 'male'], dtype=object)

In [62]:
le.inverse_transform([0,1,1,0,0,1,1])

array(['female', 'male', 'male', 'female', 'female', 'male', 'male'],
      dtype=object)

In [67]:
le.fit_transform(train['Embarked'])
# NaN값이 포함되어 있다면 LabelEncoder가 정상적으로 동작하지 않음

array([2, 0, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0,
       1, 2, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
       2, 0, 2, 2, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2, 0, 1, 2, 0, 2, 0, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1,
       2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 1, 2, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 0,
       2, 2, 2, 1, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1,

In [64]:
le.classes_

array(['C', 'Q', 'S'], dtype=object)